# IT420 - Lab Setup - Notes
#### August 2021

````
Using the DetectionLab Vagrant Files to build a DC, WEF, Logger and Win10 Box
Customizing the Vagrant File to also build the qdPM box
````

# <a id='index'></a>
## Table of Contents

[DetectionLab Vagrant Usage](#detectionlab)

[Router or Firewall Setup for Port Forwarding](#routerfirewall)

[Ubuntu 18.04 - qdPM 9.1](#qdpm)<br />
>[Objectives](#uobjectives)<br />
>[Notes from Installation](#unotes)<br />
>[Attack Path](#uattackpath)

[Ubuntu 18.04 -Piwigo 11.5.0](#piwigo)<br /> 

[Windows Server 2016 Domain Controller](#win2016)<br />
>[Notes from Installation](#wsnotes)<br />
>[Powershell Script to Create User Accounts on DC](#wsps)

[Windows 10A](#win10A)<br />
>[Notes from Setup](#w10anotes)<br />
>[Build Vulnerable Windows Service](#w10avulnservice)<br />
>[Attack Path](#w10aattackpath)

<a id='detectionlab'></a>
## DetectionLab Vagrant Usage (No longer using...)

[Back to Table of Contents](#index)

````
To setup all of the boxes for the first time (Currently stops on the windows enabling shared folders.  Install vmware tools manually and it passes the vagrant reload <machine> --provision.
$ vagrant up
# This will download the new images...  It will read the whole VagrantFile and setup 5 boxes

# To bring up and build the machine the first time
$ vagrant up dc --provision

# To reload any changes made to the Vagrant file
$ vagrant reload dc --provision
# Fails to enable shared folders - Reboot dc - Install VM Tools and restart with the above command

$ vagrant up wef --provision
$ vagrant up win10 --provision
$ vagrant up logger --provision
$ vagrant up qdPM --provision

# Removed osquery and fleet from installing...  
# Removed the redteam tools that are installed...

# After ovf transfer you need to disable the network adaptor that is NATed
# Then login as vagrant and look at /etc/netplan/01-netcfg and add the nameservers to 50-vagrant then delete 01-netcfg

````

<a id='routerfirewall'></a>
## Router or Firewall Setup for Port Forwarding

[Back to Table of Contents](#index)

In [1]:
%%html
<style>
table {align:left;display:block}
</style>

<!-- Change the style of tables to left align -->

#### Port Forwarding Configuration
External Facing IP Address (EFI)

|Port Forwarded|VM IP|Description|Notes|
|:---|:---:|:---|:--|
|EFI:22|192.168.38.50|qdPM SSH (Web Server)||
|EFI:80|192.168.38.50|qdPM (Web Server)||
|EFI:443|192.168.38.103|wef (Server 2016 - ATA)|*HTTPS|
|EFI:8000|192.168.38.105|Splunk (Logger)|*Add logger in /etc/hosts with EFI *https|
|EFI:8080|192.168.38.105|Guacamole (Logger)||
|EFI:8081|192.168.38.60|Piwigo (Web Server)||
|EFI:9999|192.168.38.105|Velociraptor (Logger)||

<br />
*Anything on logger is meant to be out-of-scope for the assessment but allowed to be access for observing what is occurring during the attacks that are occurring. 

<a id='qdpm'></a>
## Ubuntu 18.04 - qdPM 9.1

[Back to Table of Contents](#index)

<a id='uobjectives'></a>
#### Objectives

1. Password Spray to gain qdPM Admin Access
2. Create a user account - Authenticate and then use an exploit to create a PHP WebShell
3. Use the webshell to find the DB Password
4. Due to password reuse SSH into box with reused database password

***Add
5. SSH pivot from Ubuntu to other computers

<a id='unotes'></a>
#### Notes from Installation

````
Networking, configured the host only network to be 192.168.38.50 for the interfaces the detection lab will communicate with...  

Enabled the bridged networking for access from my laptop to the interface for qdpm... 10.27.20.63

Downloaded vulnerable application from: https://www.exploit-db.com/exploits/50175
Installed apache2, php7, php-xml and mysql

SSH (Create the following accounts)
Username: vagrant  Password: vagrant (Meant to be used for defense)
Username: bodgen   Password: qdDBPassword7

MySQL

Build database called qdpm
- CREATE DATABASE qdpm;

Establish user account for the database to use
- CREATE USER 'qd'@'localhost' IDENTIFIED BY 'qdDBPassword7';
- GRANT ALL PRIVILEGES ON qdpm.* TO 'qd'@'localhost';
- FLUSH PRIVILEGES;

qd    Password: qdDBPassword7
Exploit to find MySQL Password: https://www.exploit-db.com/exploits/50176

qdPM - Admin
brailee.ogden@windomain.local  Password: Winter2022

Additional qdPM Users were created with passwords taken from the rockyou.txt file...
````

<a id='uattackpath'></a>
#### Ubuntu Attack Path

<u>Initial Connection (VPN)</u>

<u>Initial Connection (At Ensign College)</u>

<u>Prep for Access to qdPM Software</u>
````
1. Add to hosts file:
10.27.20.63      qdpm.windomain.local
````

<u>Identify User Accounts</u>
````
1. Go to ensign.edu and identify who exists on the website
2. Discuss username architecture firstinital.lastname, firstname.lastname, firstinitial concat lastname, firstname concat last initial, etc. "@windomain.local"
3. Build username list from the website (Observe that it requires email addresses)
   - Frank Flores
   - Brailee Ogden
   - Dana Situmorang
   - Pam Hirwa
````

<u>Identify Publicly Available Vulnerabilities for qdPM</u>
````
1. Search exploit-db for qdPM 9.1 exploits
   - Identify that windomain.local is the domain name of the email address
   - Find the ones I added
2. Use the forgot password page of qdPM to identify the usernames/emails that are active
````

<u>Generate Password List or use Rockyou Password List</u>
````
1. Top 100 Passwords Used
2. Create <season><year> combinations etc.
````

<u>Logon as brailee.ogden@windomain.local</u>
````
1. Study the exploit on how it works... brailee.ogden@windomain.local is the admin, unable to use her account.  Add an account and use it for the exploit
2. With new user (joe.reid@windomain.local Password: Test1234!) created use the exploit to drop a PHP backdoor

---
./v2.py -url http://qdpm/ -u joe.reid@windomain.local -p Test1234!
You are not able to use the designated admin account because they do not have a myAccount page.

Backdoor uploaded at - > http://qdpm.windomain.local//uploads/users/825599-backdoor.php?cmd=whoami
````

<u>Look Around for Additional Credentials</u>
````
1. Find additional vulnerabilities or exploits from exploitdb to find the DB Password
2. Identify the users from the /etc/passwd file
3. Login with SSH using the usernames and passwords discovered
   - Discover the bogden user from the /etc/passwd file (PrivEsc with SUID in Attack Path #3)
   - Use the database password found in the exploitdb exploit to see the mysql password
   - Discover the vagrant user from the /etc/passwd file
   - Default password for vagrant is vagrant...
5. Verify that the SSH access drops you on the web server with the uploaded backdoor
````

<u>SSH Access as vagrant</u>
````
1. https://airman604.medium.com/9-ways-to-backdoor-a-linux-box-f5f83bae5a3c
2. HTTP Exfiltration using PHP backdoor and the above link for ideas
3. Setup Metasploit for C2 with the linux box
````

#### SSH Pivoting
````
Local Pivot
ssh -L 23389:192.168.38.100:3389 bogden@10.27.20.56
1. Login with Remina with Brailee Ogdens account
2. Domain is in the URL provided...
3. RDP works, start menu does not work... (Forces them to use the command line or other methods of access)

SOCKS Proxy Pivot with -D
proxychains (TCP good) is slow, be specific on ports scanned...

proxychains msfconsole
use auxiliary/scanner/smb/smb_login
````

#### Escalation of Privileges

Vagrant account has a poor password of vagrant
After logging in as the vagrant account, you can sudo
With sudo privileges create your own user account...

#### Persistance

HTTP as exfil...  Metasploit HTTP Backdoor

<a id='piwigo'></a>
## Ubuntu 18.04 - piwigo 11.5.0

[Back to Table of Contents](#index)

#### Notes from Installation

sudo apt install apache2 php php-mysql mysql-server mysql-client unzip php-gd

Downloaded piwigo 11.5.0 from the website

Build database called piwigo 
- CREATE DATABASE piwigo;

Establish user account for the database to use
- CREATE USER 'pw'@'localhost' IDENTIFIED BY 'L0st1nSpace!';
- CREATE USER 'pw'@'10.1.0.11' IDENTIFIED BY 'L0st1nSpace!';
- GRANT ALL PRIVILEGES ON piwigo.* TO 'pw'@'localhost';
- GRANT ALL PRIVILEGES ON piwigo.* TO 'pw'@'10.1.0.11';
- FLUSH PRIVILEGES;

Changed the Apache2 default port HTTP 80 to port 8000 so it can be port forwarded from the cisco router
- Modify /etc/apache2/sites-available site:8000 
- Modify /etc/apache2/ports.conf change 80 to 8000
- Restart the apache2 service

Piwigo Installation
- Username: Admin    Password: Ch3ckEngineLight!

Enable MySQL Logging to see the queries
- Modify /etc/mysql/mysql.conf.d/mysqld.cnf
- Uncomment the general query log

#### Notes for Setup of Piwigo

Created a user account for brailee.ogden Password: MyGall3ry!!!

Vulnerability
- The login to the account is identified due to credential stuffing (using compromised creds)
- Drop the creds on a pastebin site to be found be the class
- Email Address Enumeration on Register of an Account (If email matches someone else it displays a message that the email is already in use)

The admin.php has an error

URL: http://192.168.38.60:8000/admin.php?page=cat

sudo mkdir _data 

sudo chmod 777 _data

sudo mkdir upload

sudo chmod 777 upload

#### Compromised List

````
JAMES.SMITH@WINDOMAIN.LOCAL:itoomwoytpevbmr
JAMES.YOUNG@WINDOMAIN.LOCAL:opizjlcrvdkzxjm
JOHN.JONES@WINDOMAIN.LOCAL:umcyoddjfrmmfow
JOHN.JACKSON@WINDOMAIN.LOCAL:thfzreeqrdpwpof
JOHN.RODRIGUEZ@WINDOMAIN.LOCAL:hjadirbadccnujd
ROBERT.JACKSON@WINDOMAIN.LOCAL:vdafqyfzksumzqr
ROBERT.WALKER@WINDOMAIN.LOCAL:jcjyggboxtzovwj
MICHAEL.MILLER@WINDOMAIN.LOCAL:tvymjugmyfluvzi
MICHAEL.CLARK@WINDOMAIN.LOCAL:owzjbpdpjneswkh
WILLIAM.BROWN@WINDOMAIN.LOCAL:fgeeaulkaynxgvt
DAVID.MILLER@WINDOMAIN.LOCAL:ljppkyzrlanzpyc
DAVID.WALKER@WINDOMAIN.LOCAL:yvtkshzvkuardbt
BRAILEE.OGDEN@WINDOMAIN.LOCAL:MyGall3ry!!!
RICHARD.JACKSON@WINDOMAIN.LOCAL:ukqpskcqdwzvjis
CHARLES.SMITH@WINDOMAIN.LOCAL:gtwplyfnxefmjud
CHARLES.RODRIGUEZ@WINDOMAIN.LOCAL:wpvvcqcyqprqirk
JOSEPH.JACKSON@WINDOMAIN.LOCAL:pdeukzpjskuurtg
THOMAS.JONES@WINDOMAIN.LOCAL:fofrrqgqgavahpl
THOMAS.YOUNG@WINDOMAIN.LOCAL:zpmkmeuxgioboap
CHRISTOPHER.CLARK@WINDOMAIN.LOCAL:jeniklbkchyomfj
DANIEL.ANDERSON@WINDOMAIN.LOCAL:yczkkmiedgyfmqj
PAUL.MILLER@WINDOMAIN.LOCAL:rpbhthhaqzovcnl
MARK.JONES@WINDOMAIN.LOCAL:qssvlubuipjxyor
DONALD.JOHNSON@WINDOMAIN.LOCAL:lcivdpozimuefua
GEORGE.SMITH@WINDOMAIN.LOCAL:oxccwttrmhhjzhy
KENNETH.SMITH@WINDOMAIN.LOCAL:xvzddoaqprdxwcu
STEVEN.JOHNSON@WINDOMAIN.LOCAL:bdpvvuwgqozqfhm
EDWARD.JONES@WINDOMAIN.LOCAL:aflsingaivfzind
MARY.MILLER@WINDOMAIN.LOCAL:cfczyxojngwkrjw
PATRICIA.ANDERSON@WINDOMAIN.LOCAL:exjrwuafgaihiei
LINDA.OGDEN@WINDOMAIN.LOCAL:vvzwqtedhzwuszc
BARBARA.CLARK@WINDOMAIN.LOCAL:pibliakhpefrmwc
ELIZABETH.YOUNG@WINDOMAIN.LOCAL:uqfpazspbqpcukj
MARIA.JONES@WINDOMAIN.LOCAL:myscfslukancrhu
SUSAN.JACKSON@WINDOMAIN.LOCAL:aglxpymnpbkdirc
````

#### Attack Path

Admin --> Plugin --> Local Files Editor  --> Settings --> Modify PHP for Backdoor

Backdoor --> Privilege Escalation with find Command --> SSH

<a id='win2016'></a>
## Windows Server 2016 Domain Controller

[Back to Table of Contents](#index)

<a id='wsnotes'></a>
#### Notes from Installation

````
Promoted to Domain Controller with DNS Server
Domain: windomain.local

Administrator
Password: Five4Fighting!

student
Password: Five4Fighting!!

IP Address: 192.168.1.160
````

<a id='wsps'></a>
#### Powershell Script to Create User Accounts on DC

<pre><code>
# Create user accounts for IT420
# August 2021

# Create 2 OUs for the user accounts
New-ADOrganizationalUnit -Name "ExecutiveBoard"
New-ADOrganizationalUnit -Name "Employees"
New-ADOrganizationalUnit -Name "ServiceAccounts"

# Create Users for Executive Board
$ou = "OU=ExecutiveBoard,DC=windomain,DC=local"
$username = "bruce.kusch"
$dname = "Bruce Kusch"
$givename = "Bruce"
$surname = "Kusch"
$pass = ConvertTo-SecureString "Winter2022" -AsPlainText -force
New-AdUser -Name $username -Path $ou -Enabled $True -ChangePasswordAtLogon $true -AccountPassword $pass -passThru -DisplayName $dname -Surname $surname -GivenName $givename

$ou = "OU=ExecutiveBoard,DC=windomain,DC=local"
$username = "tim.sloan"
$dname = "Tim Sloan"
$givename = "Tim"
$surname = "Sloan"
$pass = ConvertTo-SecureString "MySocks!3" -AsPlainText -force
New-AdUser -Name $username -Path $ou -Enabled $True -ChangePasswordAtLogon $false -AccountPassword $pass -passThru -DisplayName $dname -Surname $surname -GivenName $givename

$ou = "OU=ExecutiveBoard,DC=windomain,DC=local"
$username = "guy.hollingsworth"
$dname = "Guy Hollingsworth"
$givename = "Guy"
$surname = "Hollingsworth"
$pass = ConvertTo-SecureString "MyRocks#3" -AsPlainText -force
New-AdUser -Name $username -Path $ou -Enabled $True -ChangePasswordAtLogon $false -AccountPassword $pass -passThru -DisplayName $dname -Surname $surname -GivenName $givename

$ou = "OU=ExecutiveBoard,DC=windomain,DC=local"
$username = "mark.richards"
$dname = "Mark Richards"
$givename = "Mark"
$surname = "Richards"
$pass = ConvertTo-SecureString "PoundSand#3" -AsPlainText -force
New-AdUser -Name $username -Path $ou -Enabled $True -ChangePasswordAtLogon $false -AccountPassword $pass -passThru -DisplayName $dname -Surname $surname -GivenName $givename

$ou = "OU=ExecutiveBoard,DC=windomain,DC=local"
$username = "alan.young"
$dname = "Alan Young"
$givename = "Alana"
$surname = "Young"
$pass = ConvertTo-SecureString "@SANSClass3" -AsPlainText -force
New-AdUser -Name $username -Path $ou -Enabled $True -ChangePasswordAtLogon $false -AccountPassword $pass -passThru -DisplayName $dname -Surname $surname -GivenName $givename


## Add Employee Accounts
$ou = "OU=Employees,DC=windomain,DC=local"
$username = "brailee.ogden"
$dname = "Brailee Ogden"
$givename = "Brailee"
$surname = "Ogden"
$pass = ConvertTo-SecureString "Winter2022" -AsPlainText -force
New-AdUser -Name $username -Path $ou -Enabled $True -ChangePasswordAtLogon $false -AccountPassword $pass -passThru -DisplayName $dname -Surname $surname -GivenName $givename

$ou = "OU=Employees,DC=windomain,DC=local"
$username = "frank.flores"
$dname = "Frank Flores"
$givename = "Frank"
$surname = "Flores"
$pass = ConvertTo-SecureString "ChangeMe!3" -AsPlainText -force
New-AdUser -Name $username -Path $ou -Enabled $True -ChangePasswordAtLogon $false -AccountPassword $pass -passThru -DisplayName $dname -Surname $surname -GivenName $givename

$ou = "OU=Employees,DC=windomain,DC=local"
$username = "dara.sitmorang"
$dname = "Dara Sitomorang"
$givename = "Dara"
$surname = "Sitomorang"
$pass = ConvertTo-SecureString "*GuitarHero*3" -AsPlainText -force
New-AdUser -Name $username -Path $ou -Enabled $True -ChangePasswordAtLogon $false -AccountPassword $pass -passThru -DisplayName $dname -Surname $surname -GivenName $givename

$ou = "OU=Employees,DC=windomain,DC=local"
$username = "pam.hirwa"
$dname = "Pam Hirwa"
$givename = "Pam"
$surname = "Hirwa"
$pass = ConvertTo-SecureString "DontChange%3" -AsPlainText -force
New-AdUser -Name $username -Path $ou -Enabled $True -ChangePasswordAtLogon $false -AccountPassword $pass -passThru -DisplayName $dname -Surname $surname -GivenName $givename

## Create the Service Account for the Lasers
$ou = "OU=ServiceAccounts,DC=windomain,DC=local"
$username = "laser.service"
$dname = "Laser Service"
$givename = "Laser"
$surname = "Service"
$pass = ConvertTo-SecureString "SuperLongP@ssw0rd,Youcantcrackme!" -AsPlainText -force
New-AdUser -Name $username -Path $ou -Enabled $True -ChangePasswordAtLogon $false -AccountPassword $pass -passThru -DisplayName $dname -Surname $surname -GivenName $givename

# SetSPN on the Laser Service Account
setspn -a HTTP/myweb laser.service

# Make the Laser Service account a member of the Domain Admins Group
Add-ADGroupMember -Identity "Domain Admins" -Members laser.service

</code></pre>

<a id='win10A'></a>
## Windows 10A

[Back to Table of Contents](#index)

<a id="w10anotes"></a>
#### Notes from Installation

[Back to Table of Contents](#index)

````
Local Admin Account
Username: vagrant
Password: vagrant

IP Address: 192.168.38.100/24 Overwrote DNS Server to make it the DC

Enabled Remote Desktop
reg add "HKEY_LOCAL_MACHINE\SYSTEM\CurrentControlSet\Control\Terminal Server" /v fDenyTSConnections /t REG_DWORD /d 0 /f

Enabled the Firewall Policies for Remote Desktop
netsh advfirewall firewall set rule group="remote desktop" new enable=Yes

Add brailee.ogden to remote desktop users group on the machine
net localgroup "Remote Desktop Users" brailee.ogden /add

Added a fileshare with authenticated users with permissions to write...  This also adjusted the windows fw
Fileshare name: ??
````

<a id="w10avulnservice"></a>
#### Build Vulnerable Service

[Back to Table of Contents](#index)

````
1. Download Process Explorer from Microsoft SysInternals Suite
2. Started the SNMP Trap Service 
3. Find the service in Process Explorer, went into properties
4. Click the services tab and then lower left click permissions
5. Added brailee.ogden to be able to have full permissions
6. With full permissions they will be able to escalate privileges to "Local Service"
7. Modified the service to run as the local system account so instead of loading as local service
````

<a id="w10aattackpath"></a>
#### Attack Path

[Back to Table of Contents](#index)

Website SSH SOCKS Proxy or Python Port Scanner

SMB Auth Scanning with CrackMapExec

SMB Auth with CrackMapExec Command Execution

Reverse Shell (?) - Windows Defender is disabled so no issue should be experienced

Privilege Escalate to Admin Privileges using the vulnerable service... Find with powerup (?)


#### SMB Auth Scanning with CrackMapExec
Setup an SSH port forward to 445...  ssh ... -L 445:192.168.1.102:445

Then use crackmapexec to auth based on a list of accounts
-  crackmapexec smb 127.0.0.1:445 -u username -p password 
-  crackmapexec smb 127.0.0.1:445 -u username username2 -p password
-  crackmapexec smb 127.0.0.1:445 -u /usr/list.txt -p /usr/pass.txt --continue-on-success

You can use crackmapexec to execute commands on the remote system
- crackmapexec smb 127.0.0.1:445 -u username -p password -x command